# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [39]:
!cat publications.tsv










2015-07-02	Do we perceive others better than ourselves? A perceptual benefit for noise-vocoded speech produced by an average speaker.	PLoS One	Tags: speech perception, phonetics, psycholinguistics	"Schuerman, W., Meyer, A. S., & McQueen, J. M. (2015). ""Do we perceive others better than ourselves? A perceptual benefit for noise-vocoded speech produced by an average speaker."" <i>PLoS One</i>, 10(7): e0129731"	paper-title-number-9	10.1371/journal.pone.0129731

## Import pandas

We are using the very handy pandas library for dataframes.

In [40]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [45]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2022-01-01,Human intracranial recordings reveal distinct ...,In Review,"Tags: vagus nerve stimulation, neuromodulation...","Schuerman, W. L., Nourski, K. V., Rhone, A. E....",paper-title-number-1,url
1,2021-09-07,Learning nonnative speech sounds changes local...,Proceedings of the National Academy of Sciences,"Tags: sound learning, ECoG","Han, G. Y., Chandrasekaran, B., Nourski, K.V.,...",paper-title-number-2,url
2,2020-09-21,Using expletive insertion to pursue and sancti...,Journal of Sociolinguistics,"Tags: taboo speech, conversation analysis","Hoey, E. M., Hömke, P., Löfgren, E., Neumann, ...",paper-title-number-3,url
3,2020-08-06,Non-invasive peripheral nerve stimulation sele...,npj Science of Learning,"Tags: sound learning, transcutaneous auricular...","Llanos, F., McHaney, J. R., Schuerman, W. L., ...",paper-title-number-4,10.1038/s41539-020-0070-0.
4,2019-05-20,Comparison of Common Artifact Rejection Method...,in 2019 9th International IEEE/EMBS Conference...,"Tags: cortical stimulation, peripheral stimula...","Sellers, K.K., Schuerman, W.L., Dawes, H. E., ...",paper-title-number-5,10.1109/NER.2019.8716980
5,2017-04-17,Sensorimotor adaptation affects perceptual com...,Journal of the Acoustical Society of America,"Tags: altered auditory feedback, compensation ...","Schuerman, W. L., Nagarajan, S., McQueen, J. M...",paper-title-number-6,url
6,2017-04-07,Mapping the Speech Code: Cortical Responses Li...,Frontiers in Human Neuroscience,"Tags: altered auditory feedback, speech percep...","Schuerman, W. L., Meyer, A., & McQueen, J. M. ...",paper-title-number-7,url
7,2016-06-01,What sound symbolism can and cannot do: testin...,Language,"Tags: phonetics, sound symbolism, psycholingui...","Dingemanse, M., Schuerman, W., Reinisch, E., T...",paper-title-number-8,url
8,2015-07-02,Do we perceive others better than ourselves? A...,PLoS One,"Tags: speech perception, phonetics, psycholing...","Schuerman, W., Meyer, A. S., & McQueen, J. M. ...",paper-title-number-9,10.1371/journal.pone.0129731


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [46]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [47]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [48]:
!ls ../_publications/

2015-07-02-paper-title-number-9.md 2020-08-06-paper-title-number-4.md
2016-06-01-paper-title-number-8.md 2020-09-21-paper-title-number-3.md
2017-04-07-paper-title-number-7.md 2021-09-07-paper-title-number-2.md
2017-04-17-paper-title-number-6.md 2022-01-01-paper-title-number-1.md
2019-05-20-paper-title-number-5.md
